In [1]:
#pip install keras-tuner --upgrade

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import datetime
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
%matplotlib inline

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout

In [3]:
def fetch_dataset(path):
    images = []
    labels = []
    for dir1 in os.listdir(path):
        for file in os.listdir(os.path.join(path,dir1)):
            image_path = os.path.join(path,dir1,file)
            with Image.open(image_path).convert('L') as img:
                arr = np.array(img.getdata()).reshape(256, 256, 1) / 255
                images.append(arr)
                labels.append(dir1)
    
    unique_labels = {k: v for v,k in enumerate(np.unique(labels))}
    numeric_labels = [unique_labels[labels[i]] for i in range(len(labels))]
    #print(numeric_labels)
    return np.array(images),numeric_labels


In [4]:
train_path = os.path.join("data","train")
val_path = os.path.join("data","val")
test_path = os.path.join("data","test")


x_train,y_train = fetch_dataset(train_path)
x_val,y_val = fetch_dataset(val_path)
#x_test,y_test = fetch_dataset(test_path)

In [5]:
y_train_classes = y_train
y_val_classes = y_val
y_train = tf.keras.utils.to_categorical(y_train_classes, 4)
y_val = tf.keras.utils.to_categorical(y_val_classes, 4)

In [6]:
classes = ["COVID", "HEALTHY", "PNEUMONIA_BAC", "PNEUMONIA_VIR"]

In [7]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [8]:
# early stopping values
callback_stop = tf.keras.callbacks.EarlyStopping(
    monitor="recall_m",
    min_delta=0.0003,
    patience=5,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True)

# Keras Tuner part

### Iteration 4

Dropout layer test + dropout rate

In [9]:
import keras_tuner as kt
from keras_tuner import Objective

# Feature Maps test
## für Moritz
Kommentare bei der search funktion weiter unten raus nehmen und laufen lassen

In [10]:
def build_model_2(hp):
    data_dims = (256, 256, 1)
    dropout = hp.Boolean("dropout")
    d_rate = hp.Float("d_rate", min_value=0.30, max_value=0.70, step=0.20)

    model = Sequential()
    model.add(Conv2D(15,kernel_size=(3, 3), activation="relu", padding="same", input_shape=data_dims))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(15, kernel_size=(3, 3), activation="relu", padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    if dropout:
        model.add(Dropout(0.30))
    model.add(Conv2D(15, kernel_size=(3, 3), activation="relu", padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    if dropout:
        model.add(Dropout(0.30))
    model.add(Conv2D(15, kernel_size=(3, 3), activation="relu", padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    if dropout:
        model.add(Dropout(0.30))
    
    model.add(Flatten())
    model.add(Dense(16, activation="sigmoid"))
    model.add(Dropout(0.25))
    model.add(Dense(4, activation="softmax"))

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", "categorical_crossentropy", f1_m, precision_m, recall_m])
    return model


In [11]:
build_model_2(kt.HyperParameters())

In [12]:
tuner = kt.RandomSearch(
    hypermodel=build_model_2,
    objective=Objective("val_recall_m", direction="max"),
    max_trials=20,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="x-ray-test"
)

Search funktion für Moritz. Hier Kommentare raus nehmen

In [13]:
# tuner.search(x_train, y_train, 
#              epochs=4, 
#              batch_size=256,
#              validation_data=(x_val, y_val), 
#              verbose=1
#         )

# Conv layer test
## für Georg
Kommentare bei der search funktion weiter unten raus nehmen und laufen lassen

In [14]:
def build_model_3(hp):
    data_dims = (256, 256, 1)
    d_rate = hp.Float("d_rate", min_value=0.30, max_value=0.70, step=0.20)

    model = Sequential()
    model.add(Conv2D(15,kernel_size=(3, 3), activation="relu", padding="same", input_shape=data_dims))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(15, kernel_size=(3, 3), activation="relu", padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    if hp.Boolean("dropout_1"):
        model.add(Dropout(0.30))
    model.add(Conv2D(15, kernel_size=(3, 3), activation="relu", padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    if hp.Boolean("dropout_2"):
        model.add(Dropout(0.30))
    model.add(Conv2D(15, kernel_size=(3, 3), activation="relu", padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    if hp.Boolean("dropout_3"):
        model.add(Dropout(0.30))
    
    model.add(Flatten())
    model.add(Dense(16, activation="sigmoid"))
    model.add(Dropout(0.25))
    model.add(Dense(4, activation="softmax"))

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", "categorical_crossentropy", f1_m, precision_m, recall_m])
    return model

In [15]:
build_model_3(kt.HyperParameters())

In [16]:
tuner = kt.RandomSearch(
    hypermodel=build_model_3,
    objective=Objective("val_recall_m", direction="max"),
    max_trials=25,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="x-ray-test_2"
)

Search funktion für Georg. Hier Kommentare raus nehmen

In [17]:
# tuner.search(x_train, y_train, 
#              epochs=4, 
#              batch_size=256,
#              validation_data=(x_val, y_val), 
#              verbose=1
#         )

# Tuner Output

In [18]:
tuner.results_summary()

Results summary
Results in my_dir\x-ray-test_2
Showing 10 best trials
Objective(name='val_recall_m', direction='max')


In [19]:
# Get the top 3 models.
models = tuner.get_best_models(num_models=3)
best_model = models[0]
second_model = models[1]
third_model = models[2]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build(input_shape=(None, 256, 256))
best_model.summary()
second_model.build(input_shape=(None, 256, 256))
second_model.summary()
third_model.build(input_shape=(None, 256, 256))
third_model.summary()

IndexError: list index out of range